In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().master("local[2]").getOrCreate()

In [ ]:
graph_schema = StructType([
    StructField("to", IntegerType(), False),
    StructField("from", IntegerType(), False)
])

In [ ]:
dist_schema = StructType([
    StructField("vertex", IntegerType(), False),
    StructField("distance", IntegerType(), False)
])

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import SQLContext
sc = spark.sparkContext



In [ ]:
def shortest_path(v_from, v_to, dataset_path=None):
    
    schema = StructType([StructField('parent',IntegerType(), False),StructField('u', IntegerType(), False)])
    parent = spark.createDataFrame(sc.emptyRDD(), schema)

    edges = spark.read.csv(dataset_path, sep="\t", schema=graph_schema)
    E = edges.select(edges.to.alias("v"), col("from").alias("u"))
    path_length = None
    
    current = [v_from]
    path_length=1
    while True:
        neighbours = E.filter(E.u.isin(current))
        visited = parent.select("u").distinct().rdd.map(lambda row : row[0])
        
        non_visited = neighbours.filter(~neighbours.v.isin(visited.collect()) & (neighbours.v!=v_from))
        parent = parent.union(non_visited.select("u","v"))
        if (parent.filter(parent.u==v_to)).count()>0:
            break
        current = non_visited.select("v").distinct().rdd.map(lambda row : row[0]).collect()
        path_length+=1
    return path_length

    


In [ ]:
print shortest_path(12, 34, "/data/twitter/twitter_sample2.txt")